In [12]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
import upload_data
from pathlib import Path

# # Set headers to mimic a real browser
# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (compatible; MySoccerScraper/1.0)"
# }

# # Step 1: Scrape the fixture list
# #fixture_url = "https://fbref.com/en/comps/9/2025-2026/schedule/2025-2026-Premier-League-Scores-and-Fixtures"
# fixture_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
# res = requests.get(fixture_url, headers=HEADERS)
# soup = BeautifulSoup(res.content, "lxml")

# # Step 2: Find the fixture table
# fixtures_table = soup.find("table", {"id": "sched_2025-2026_9_1"})


with open("match_links/match_links.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "lxml")

# Find your table (adjust the id as needed)
fixtures_table = soup.find("table", {"id": "sched_2025-2026_9_1"})

fixtures_df = pd.read_html(str(fixtures_table))[0]
#first filtering out the matches that haven't been played
fixtures_df = fixtures_df[fixtures_df['Match Report'] == 'Match Report']
#then filtering out rows that have the headers repeated like they do in fbref
fixtures_df = fixtures_df[fixtures_df['Score'] != 'Score']

match_join_info = {"home": [], "away": [], "match_id": []}
match_id_list = []
for row in fixtures_table.find_all("tr"):
    #finding match report link
    match_report_link = row.find("td", {"data-stat": "match_report"})
    gameweek = row.find('th', {'data-stat': 'gameweek'}).text
    if match_report_link and row.find('a', href=True):
        if match_report_link.find("a").text == 'Match Report':
            match_link = match_report_link.find("a")["href"]
            fixtures_match_id = match_link.split(('/'))[5]
            match_id_list.append(fixtures_match_id)

fixtures_df['match_id'] = match_id_list
fixtures_df.head()
fixtures_df.to_csv("fixtures_df.csv")
upload_data.upload_fixtures(fixtures_df)

Pandas(Wk='1', Day='Fri', Date='2025-08-15', Time='20:00 (15:00)', Home='Liverpool', xG='2.2', Score='4–2', _7='1.7', Away='Bournemouth', Attendance='60315', Venue='Anfield', Referee='Anthony Taylor', _12='Match Report', Notes=nan, match_id='a071faa8') inserted into fixtures
Pandas(Wk='1', Day='Sat', Date='2025-08-16', Time='12:30 (07:30)', Home='Aston Villa', xG='0.2', Score='0–0', _7='1.4', Away='Newcastle Utd', Attendance='42526', Venue='Villa Park', Referee='Craig Pawson', _12='Match Report', Notes=nan, match_id='bbdf4739') inserted into fixtures
Pandas(Wk='1', Day='Sat', Date='2025-08-16', Time='15:00 (10:00)', Home='Sunderland', xG='0.7', Score='3–0', _7='0.6', Away='West Ham', Attendance='46233', Venue='Stadium of Light', Referee='Robert Jones', _12='Match Report', Notes=nan, match_id='e1058522') inserted into fixtures
Pandas(Wk='1', Day='Sat', Date='2025-08-16', Time='15:00 (10:00)', Home='Brighton', xG='1.5', Score='1–1', _7='0.7', Away='Fulham', Attendance='31478', Venue='The

In [13]:
fixtures_df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,match_id
0,1,Fri,2025-08-15,20:00 (15:00),Liverpool,2.2,4–2,1.7,Bournemouth,60315,Anfield,Anthony Taylor,Match Report,NaN,a071faa8
1,1,Sat,2025-08-16,12:30 (07:30),Aston Villa,0.2,0–0,1.4,Newcastle Utd,42526,Villa Park,Craig Pawson,Match Report,NaN,bbdf4739
2,1,Sat,2025-08-16,15:00 (10:00),Sunderland,0.7,3–0,0.6,West Ham,46233,Stadium of Light,Robert Jones,Match Report,NaN,e1058522
3,1,Sat,2025-08-16,15:00 (10:00),Brighton,1.5,1–1,0.7,Fulham,31478,The American Express Stadium,Samuel Barrott,Match Report,NaN,57c49bae
4,1,Sat,2025-08-16,15:00 (10:00),Tottenham,2.3,3–0,0.9,Burnley,61077,Tottenham Hotspur Stadium,Michael Oliver,Match Report,NaN,d41bb8b3


In [14]:
#Step 4: Match team name with corresponding team id:
pl_teams = {"Arsenal":"18bb7c10", "Aston Villa": "8602292d", "Bournemouth": "4ba7cbea", "Brighton": "d07537b9", "Burnley": "943e8050", 
"Brentford": "cd051869", "Chelsea": "cff3d9bb", "Crystal Palace": "47c64c55", "Everton": "d3fd31cc", "Fulham": "fd962109", "Ipswich Town": "b74092de", 
"Leicester City": "a2d435b3", "Leeds United": "5bfb9659", "Liverpool": "822bd0ba", "Manchester City": "b8fd03ef", "Manchester Utd": "19538871", 
"Newcastle Utd": "b2b47a98", "Norwich City": "1c781004", "Nott'ham Forest": "e4a775cb", "Sheffield Utd": "1df6b87e", "Southampton": "33c895d4", 
"Sunderland": "8ef52968", "Tottenham": "361ca564", "West Brom": "60c6b05f", "West Ham":"7c21e445", "Wolves": "8cec06e1", "Watford": "2abfe087"}


In [15]:
#Step 6: Renaming columns for better understanding
def rename_cols(stat_type, df):
    if stat_type == "summary":
        df = df.rename(columns={"Cmp": "Passes_cmp", "Att": "Passes_att", "Cmp%": "Passes_cmp_pct", "Att.1": "Take-ons_att", "Succ":"Take-ons_succ"})
    if stat_type == "passing":
        df = df.rename(columns={"Cmp": "Total_passes_cmp", "Att": "Total_passes_att", "Cmp%": "Total_passes_cmp%", "TotDist": "TotPassDist", "PrgDist": "PrgPassDist",
        "Cmp.1": "Short_passes_cmp", "Att.1": "Short_passes_att", "Cmp%.1": "Short_passes_cmp%", "Cmp.2": "Med_passes_cmp", "Att.2": "Med_passes_att", "Cmp%.2": "Med_passes_cmp%",
        "Cmp.3": "Long_passes_cmp", "Att.3": "Long_passes_att", "Cmp%.3": "Long_passes_cmp%", 'KP': "key_passes", '1/3': "passes_into_final_third", 
        'PPA': "passes_into_pen_area", 'CrsPA':"crosses_into_pen_area" ,'PrgP': "progressive_passes"})
    if stat_type == "passing_types":
        df = df.rename(columns={'Live':"live_ball_passes", 'Dead':"dead_ball_passes", 'FK':'free_kick_passes', 'TB':'through_balls', 'Sw':'switches', 'Crs':'crosses', 
        'TI':'throw_ins', 'CK': 'corner_kicks', 'In': 'inswinging_corner', 'Out': 'outswinging_corner', 'Str': 'straight_corner', 'Cmp': 'passes_completed', 
        'Off': 'passes_offside', 'Blocks': 'passes_blocked'})
    if stat_type == "defense":
        df = df.rename(columns={"Tkl": "Total_tkls", "TklW": "Total_tkls_won", "Tkl.1": "Dribbler_tkls", "Att": "Dribbler_tkls_att", "Tkls%": "Dribbler_tkls%_won", 
        "Lost": "Dribbler_tkls_lost", "Sh": "Shots_blocked", "Pass": "Passes_blocked"})
    if stat_type == "possession":
        df = df.rename(columns={'Touches':"Total_touches",'Def Pen': "Def_pen_touches", 'Def 3rd': "Def_3rd_touches", 'Mid 3rd': "Mid_3rd_touches", 
        'Att 3rd': "Att_3rd_touches", 'Att Pen': "Att_pen_touches", 'Live': "Live_touches", 'Att': "take_ons_att", 'Succ': "take_ons_succ", 'Succ%': "take_ons_succ%",
        'Tkld': "Tkld_during_take_ons", 'Tkld%': "Tkld%_during_take_ons", "Rec": "Passes_received", "PrgR": "Progressive_passes_received"})
    if stat_type == "misc":
        df = df.rename(columns={'CrdY':'yellow_card','CrdR':'red_card', '2CrdY':'two_yellow_cards', 'Fls': 'fouls_committed', 'Fld':'fouls_drawn', 'Off':'offsides', 
        'Crs': 'Crosses', 'TklW': 'Tkls_won','PKcon': "PK_conceded", 'OG': 'own_goals', 'Recov':'ball_recoveries', 'Won': "Aerial_Duels_won", 'Lost': "Aerial_Duels_lost", 
        'Won%': "Aerial_Duels_won%"})
    return df
    

In [16]:
#All games
folder = Path("games_html")

# Iterate through all HTML files in the folder
for html_file in folder.glob("*.html"):
    print(html_file)
    with open(html_file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "lxml")
        # Find all HTML comments
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))

        # Getting the URL so we can get the match id and then join with the previous table to get the home and away team
        url = None
        for comment in comments:
            if "saved from url=" in comment:
                # Extract URL using string splitting
                start = comment.find("http")
                end = comment.find(">", start)
                url = comment[start:end].strip(" >")
                break

        match_id = str(url.split(('/'))[5])
        print(match_id)
        home_team = fixtures_df.query(f"match_id == '{match_id}'")["Home"].unique()[0]
        away_team = fixtures_df.query(f"match_id == '{match_id}'")["Away"].unique()[0]
        home_team_id = pl_teams[home_team]
        away_team_id = pl_teams[away_team]
        #Extracting Player IDs for primary key
        home_table = soup.find("table", {"id": f"stats_{home_team_id}_summary"})
        away_table = soup.find("table", {"id": f"stats_{away_team_id}_summary"})
        home_player_ids = [th['data-append-csv'] for th in home_table.find_all('th', attrs={'data-append-csv': True})]
        away_player_ids = [th['data-append-csv'] for th in away_table.find_all('th', attrs={'data-append-csv': True})]

        #Extracting Summary Stats
        home_team_summary = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_summary"})), header=1)
        home_team_summary = home_team_summary[0].iloc[0:len(home_team_summary[0])-1]
        away_team_summary = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_summary"})), header=1)
        away_team_summary = away_team_summary[0].iloc[0:len(away_team_summary[0])-1]
        home_team_summary['team'] = home_team
        home_team_summary['opponent'] = away_team
        away_team_summary['team'] = away_team
        away_team_summary['opponent'] = home_team
        summary = pd.concat([home_team_summary, away_team_summary], ignore_index=True)
        summary['player_ids'] = home_player_ids + away_player_ids
        summary['match_id'] = match_id
        summary = rename_cols("summary", summary)

        #Extracting Passing Stats
        home_team_passing = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_passing"})), header=1)
        home_team_passing = home_team_passing[0].iloc[0:len(home_team_passing[0])-1]
        away_team_passing = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_passing"})), header=1)
        away_team_passing = away_team_passing[0].iloc[0:len(away_team_passing[0])-1]
        home_team_passing['team'] = home_team
        home_team_passing['opponent'] = away_team
        away_team_passing['team'] = away_team
        away_team_passing['opponent'] = home_team
        passing = pd.concat([home_team_passing, away_team_passing], ignore_index=True)
        passing['player_ids'] = home_player_ids + away_player_ids
        passing['match_id'] = match_id
        passing = rename_cols("passing", passing)

        #Defensive Actions Stats
        home_team_defense = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_defense"})), header=1)
        home_team_defense = home_team_defense[0].iloc[0:len(home_team_defense[0])-1]
        away_team_defense = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_defense"})), header=1)
        away_team_defense = away_team_defense[0].iloc[0:len(away_team_defense[0])-1]
        home_team_defense['team'] = home_team
        home_team_defense['opponent'] = away_team
        away_team_defense['team'] = away_team
        away_team_defense['opponent'] = home_team
        defense = pd.concat([home_team_defense, away_team_defense], ignore_index=True)
        defense['player_ids'] = home_player_ids + away_player_ids
        defense['match_id'] = match_id
        defense['team'] = home_team
        defense['opposition'] = away_team
        defense = rename_cols("defense", defense)

        #Extracting Pass Type Stats
        home_team_passing_types = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_passing_types"})), header=1)
        home_team_passing_types = home_team_passing_types[0].iloc[0:len(home_team_passing_types[0])-1]
        away_team_passing_types = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_passing_types"})), header=1)
        away_team_passing_types = away_team_passing_types[0].iloc[0:len(away_team_passing_types[0])-1]
        home_team_passing_types['team'] = home_team
        home_team_passing_types['opponent'] = away_team
        away_team_passing_types['team'] = away_team
        away_team_passing_types['opponent'] = home_team
        passing_types = pd.concat([home_team_passing_types, away_team_passing_types], ignore_index=True)
        defense['player_ids'] = home_player_ids + away_player_ids
        passing_types['match_id'] = match_id
        passing_types['team'] = home_team
        passing_types['opposition'] = away_team
        passing_types = rename_cols("passing_types", passing_types)

        #Extracting Possession Stats
        home_team_possession = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_possession"})), header=1)
        home_team_possession = home_team_possession[0].iloc[0:len(home_team_possession[0])-1]
        away_team_possession = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_possession"})), header=1)
        away_team_possession = away_team_possession[0].iloc[0:len(away_team_possession[0])-1]
        home_team_possession['team'] = home_team
        home_team_possession['opponent'] = away_team
        away_team_possession['team'] = away_team
        away_team_possession['opponent'] = home_team
        possession = pd.concat([home_team_possession, away_team_possession], ignore_index=True)
        possession['player_ids'] = home_player_ids + away_player_ids
        possession['match_id'] = match_id
        possession['team'] = home_team
        possession['opposition'] = away_team
        possession = rename_cols("possession", possession)

        #Extracting Miscellaneous Stats
        home_team_misc = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_misc"})), header=1)
        home_team_misc = home_team_misc[0].iloc[0:len(home_team_misc[0])-1]
        away_team_misc = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_misc"})), header=1)
        away_team_misc = away_team_misc[0].iloc[0:len(away_team_misc[0])-1]
        home_team_misc['team'] = home_team
        home_team_misc['opponent'] = away_team
        away_team_misc['team'] = away_team
        away_team_misc['opponent'] = home_team
        misc = pd.concat([home_team_misc, away_team_misc], ignore_index=True)
        misc['player_ids'] = home_player_ids + away_player_ids
        misc['match_id'] = match_id
        misc['team'] = home_team
        misc['opposition'] = away_team
        misc = rename_cols("misc", misc)

        final_df = {"summary": summary, "passing": passing, "defense": defense, "passing_types": passing_types, "possession": possession, "misc": misc}
        for stat,df in final_df.items():
            upload_data.upload_data(df, stat)
            df.to_csv(f'{stat}.csv', index = False)


games_html/West Ham United vs. Tottenham Hotspur Match Report – Saturday September 13, 2025 _ FBref.com.html
86c7261a
Table `summary` created and data inserted.
Table `passing` created and data inserted.
Table `defense` created and data inserted.
Table `passing_types` created and data inserted.
Table `possession` created and data inserted.
Table `misc` created and data inserted.
games_html/Everton vs. Aston Villa Match Report – Saturday September 13, 2025 _ FBref.com.html
a624dc89
Table `summary` created and data inserted for match a624dc89
Table `passing` created and data inserted for match a624dc89
Table `defense` created and data inserted for match a624dc89
Table `passing_types` created and data inserted for match a624dc89
Table `possession` created and data inserted for match a624dc89
Table `misc` created and data inserted for match a624dc89
games_html/Liverpool vs. Arsenal Match Report – Sunday August 31, 2025 _ FBref.com.html
b0005978
Table `summary` created and data inserted for

In [17]:
# file_name ="games_html/Liverpool vs. Bournemouth Match Report – Friday August 15, 2025 _ FBref.com.html"
# with open(file_name, "r", encoding="utf-8") as f:
#     soup = BeautifulSoup(f, "lxml")
#     # Find all HTML comments
#     comments = soup.find_all(string=lambda text: isinstance(text, Comment))

#     # Getting the URL so we can get the match id and then join with the previous table to get the home and away team
#     url = None
#     for comment in comments:
#         if "saved from url=" in comment:
#             # Extract URL using string splitting
#             start = comment.find("http")
#             end = comment.find(">", start)
#             url = comment[start:end].strip(" >")
#             break

#     match_id = str(url.split(('/'))[5])
#     print(match_id)
#     home_team = fixtures_df.query(f"match_id == '{match_id}'")["Home"].unique()[0]
#     away_team = fixtures_df.query(f"match_id == '{match_id}'")["Away"].unique()[0]
#     home_team_id = pl_teams[home_team]
#     away_team_id = pl_teams[away_team]
#     #Extracting Player IDs for primary key
#     home_table = soup.find("table", {"id": f"stats_{home_team_id}_summary"})
#     away_table = soup.find("table", {"id": f"stats_{away_team_id}_summary"})
#     home_player_ids = [th['data-append-csv'] for th in home_table.find_all('th', attrs={'data-append-csv': True})]
#     away_player_ids = [th['data-append-csv'] for th in away_table.find_all('th', attrs={'data-append-csv': True})]

#     #Extracting Summary Stats
#     home_team_summary = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_summary"})), header=1)
#     home_team_summary = home_team_summary[0].iloc[0:len(home_team_summary[0])-1]
#     away_team_summary = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_summary"})), header=1)
#     away_team_summary = away_team_summary[0].iloc[0:len(away_team_summary[0])-1]
#     summary = pd.concat([home_team_summary, away_team_summary], ignore_index=True)
#     summary['player_ids'] = home_player_ids + away_player_ids
#     summary['match_id'] = match_id
#     summary = rename_cols("summary", summary)

#     #Extracting Passing Stats
#     home_team_passing = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_passing"})), header=1)
#     home_team_passing = home_team_passing[0].iloc[0:len(home_team_passing[0])-1]
#     away_team_passing = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_passing"})), header=1)
#     away_team_passing = away_team_passing[0].iloc[0:len(away_team_passing[0])-1]
#     passing = pd.concat([home_team_passing, away_team_passing], ignore_index=True)
#     passing['player_ids'] = home_player_ids + away_player_ids
#     passing['match_id'] = match_id
#     passing = rename_cols("passing", passing)

#     #Defensive Actions Stats
#     home_team_defense = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_defense"})), header=1)
#     home_team_defense = home_team_defense[0].iloc[0:len(home_team_defense[0])-1]
#     away_team_defense = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_defense"})), header=1)
#     away_team_defense = away_team_defense[0].iloc[0:len(away_team_defense[0])-1]
#     defense = pd.concat([home_team_defense, away_team_defense], ignore_index=True)
#     defense['player_ids'] = home_player_ids + away_player_ids
#     defense['match_id'] = match_id
#     defense = rename_cols("defense", defense)

#     #Extracting Pass Type Stats
#     home_team_passing_types = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_passing_types"})), header=1)
#     home_team_passing_types = home_team_passing_types[0].iloc[0:len(home_team_passing_types[0])-1]
#     away_team_passing_types = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_passing_types"})), header=1)
#     away_team_passing_types = away_team_passing_types[0].iloc[0:len(away_team_passing_types[0])-1]
#     passing_types = pd.concat([home_team_passing_types, away_team_passing_types], ignore_index=True)
#     defense['player_ids'] = home_player_ids + away_player_ids
#     passing_types['match_id'] = match_id
#     passing_types = rename_cols("passing_types", passing_types)

#     #Extracting Possession Stats
#     home_team_possession = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_possession"})), header=1)
#     home_team_possession = home_team_possession[0].iloc[0:len(home_team_possession[0])-1]
#     away_team_possession = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_possession"})), header=1)
#     away_team_possession = away_team_possession[0].iloc[0:len(away_team_possession[0])-1]
#     possession = pd.concat([home_team_possession, away_team_possession], ignore_index=True)
#     possession['player_ids'] = home_player_ids + away_player_ids
#     possession['match_id'] = match_id
#     possession = rename_cols("possession", possession)

#     #Extracting Miscellaneous Stats
#     home_team_misc = pd.read_html(str(soup.find("table", {"id": f"stats_{home_team_id}_misc"})), header=1)
#     home_team_misc = home_team_misc[0].iloc[0:len(home_team_misc[0])-1]
#     away_team_misc = pd.read_html(str(soup.find("table", {"id": f"stats_{away_team_id}_misc"})), header=1)
#     away_team_misc = away_team_misc[0].iloc[0:len(away_team_misc[0])-1]
#     misc = pd.concat([home_team_misc, away_team_misc], ignore_index=True)
#     misc['player_ids'] = home_player_ids + away_player_ids
#     misc['match_id'] = match_id
#     misc = rename_cols("misc", misc)

#     final_df = {"summary": summary, "passing": passing, "defense": defense, "passing_types": passing_types, "possession": possession, "misc": misc}
#     for stat,df in final_df.items():
#         upload_data.upload_data(df, stat)
#         df.to_csv(f'{stat}.csv', index = False)
